In [ ]:
import numpy as np
import pygame
import random

# Pygame initialization
WIDTH, HEIGHT = 800, 600
GRID_SIZE = 20
ROWS, COLS = HEIGHT // GRID_SIZE, WIDTH // GRID_SIZE

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
AGENT_COLOR = (0, 255, 0)
TARGET_COLOR = (255, 0, 0)

pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("SwarmBot ACO Simulation")
clock = pygame.time.Clock()

class Cell:
    def __init__(self):
        self.pheromone = 0.0
        self.target = False

grid = [[Cell() for _ in range(COLS)] for _ in range(ROWS)]

# Random target placement
for _ in range(10):
    x, y = random.randint(0, ROWS - 1), random.randint(0, COLS - 1)
    grid[x][y].target = True

class SwarmBot:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def move(self):
        neighbors = self.get_neighbors()
        if not neighbors:
            return

        weights = []
        for nx, ny in neighbors:
            pheromone = grid[nx][ny].pheromone
            weights.append(pheromone + 1.0)

        total = sum(weights)
        probabilities = [w / total for w in weights]
        choice = random.choices(neighbors, weights=probabilities, k=1)[0]

        self.x, self.y = choice
        self.lay_pheromone()

    def get_neighbors(self):
        dirs = [(-1,0), (1,0), (0,-1), (0,1)]
        neighbors = []
        for dx, dy in dirs:
            nx, ny = self.x + dx, self.y + dy
            if 0 <= nx < ROWS and 0 <= ny < COLS:
                neighbors.append((nx, ny))
        return neighbors

    def lay_pheromone(self):
        grid[self.x][self.y].pheromone += 0.5

agents = [SwarmBot(random.randint(0, ROWS-1), random.randint(0, COLS-1)) for _ in range(20)]

def draw():
    for i in range(ROWS):
        for j in range(COLS):
            rect = pygame.Rect(j*GRID_SIZE, i*GRID_SIZE, GRID_SIZE, GRID_SIZE)
            if grid[i][j].target:
                pygame.draw.rect(screen, TARGET_COLOR, rect)
            elif grid[i][j].pheromone > 0:
                alpha = min(int(grid[i][j].pheromone * 50), 255)
                pygame.draw.rect(screen, (0, 0, alpha), rect)
            else:
                pygame.draw.rect(screen, WHITE, rect)

            pygame.draw.rect(screen, BLACK, rect, 1)

    for agent in agents:
        rect = pygame.Rect(agent.y*GRID_SIZE, agent.x*GRID_SIZE, GRID_SIZE, GRID_SIZE)
        pygame.draw.rect(screen, AGENT_COLOR, rect)

running = True
while running:
    screen.fill(WHITE)
    draw()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    for agent in agents:
        agent.move()

    for row in grid:
        for cell in row:
            cell.pheromone *= 0.99  # Evaporation

    pygame.display.flip()
    clock.tick(10)

pygame.quit()
